# Sarter Notebook

In [2]:
import pandas as pd
import numpy as np

## Reading Data

In [3]:
# read train data
df_orders = pd.read_csv("orders.csv", sep=";")
df_products = pd.read_csv("product_attributes.csv", sep=",")
df_dists = pd.read_csv("cities_data.csv", sep=";")

# read test data
df_test = pd.read_csv("test.csv", sep=";")

## Dummy Model

In [4]:
from sklearn.dummy import DummyClassifier

X_train = df_orders.drop(['late_order'], axis=1)
y_train = df_orders['late_order']

X_test = df_test

dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(X_train, y_train)

pred_proba = dummy_clf.predict_proba(X_test)

## Save Solution
> **Note:** Submited probabilities must be for the **True** cases.

In [5]:
submission = pd.DataFrame({"order_id": X_test.order_id, "late_order": pred_proba[:,1]})
submission.to_csv("submission_kaggle.csv", index=False)

## Data insights

In [87]:
df_products['weight']

0      2876
1      2785
2      2751
3      2736
4      2707
       ... 
767     162
768     161
769     153
770     139
771     136
Name: weight, Length: 772, dtype: int64

In [9]:
import matplotlib.pyplot as plt

In [45]:
units = df_orders['units']

In [66]:
products = list(df_orders['product_id'].values)

In [88]:
weight = []
material_handling = []
for p in products:
    w = list(df_products.loc[df_products['product_id'] == p]['weight'].values)
    if (w == []):
        w = None
    else:
        w = w[0]
    weight.append(w)
    w = list(df_products.loc[df_products['product_id'] == p]['material_handling'].values)
    if (w == []):
        w = None
    else:
        w = w[0]
    material_handling.append(w)

In [93]:
df_orders['weight'] = weight
df_orders['material_handling'] = material_handling

In [97]:
df_orders['weight'].isnu.sum()

AttributeError: 'Series' object has no attribute 'isnan'